In [1]:
import import_ipynb
import metrics
import data_transformation
import train_validation
import model_architecture
import unittest

import os
import pandas as pd
import matplotlib.pyplot as plt
import time 
import math
from random import shuffle
import torch
import numpy as np

from torchvision import transforms, utils
from torch import nn 
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

import random
import itertools
import sys
import re

importing Jupyter notebook from metrics.ipynb
importing Jupyter notebook from data_transformation.ipynb
importing Jupyter notebook from train_validation.ipynb
importing Jupyter notebook from model_architecture.ipynb


In [2]:
def test_modules(module_test):
    suite = unittest.TestLoader().loadTestsFromModule(module_test)
    unittest.TextTestRunner(verbosity=1).run(suite)

In [3]:
testmodules = [data_transformation.TestNotebook(), metrics.metrics_test()]
output_test = []
for test in testmodules:
    test_modules(test)

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK
...
----------------------------------------------------------------------
Ran 3 tests in 0.039s

OK


In [7]:
params = {
    'batch_size': 32,
    'save_model_path': "test.chkpt",
    'epoch': 4
}

In [5]:
#load data
X_train = np.random.randint(0, 2, (320, 30, 20))
y_train = np.random.uniform(0, 1, (320, 1))
X_test = np.random.randint(0, 2, (80, 30, 20))
y_test = np.random.uniform(0, 1, (80, 1))
#make dataset? or in data_transformation
model = model_architecture.convnet(np.size(X_train, 1), np.size(X_train, 2))

pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Model parameters: "+str(pytorch_total_params) )
if torch.cuda.device_count() > 0:
    print('Using GPU' + str(pick_gpu_lowest_memory()))
    device = 'cuda:'+str(pick_gpu_lowest_memory())
else:
    print('Using CPU')
    device = 'cpu'
model = model.to(device)
criterion = metrics.select_criterion('MSE')
optimizer = optim.Adam(
        filter(lambda x: x.requires_grad, model.parameters()),
        betas=(0.9, 0.98), eps=1e-09, weight_decay = 0.0)



train_data = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train))
test_data = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test))

train_dataloader = torch.utils.data.DataLoader(train_data,
                              batch_size = params['batch_size'],
                              shuffle = True, 
                              drop_last = True)
eval_dataloader = torch.utils.data.DataLoader(test_data,
                              batch_size = params['batch_size'],
                              shuffle = True,
                              drop_last = True)

Model parameters: 217364
Using CPU


In [6]:
valid_losss, train_losses, valid_accus, train_accus = train_validation.start_training(params['save_model_path'], params['epoch'], model,
                                                                                      train_dataloader, eval_dataloader, optimizer,
                                                                                      device, criterion)

[ Epoch 0 ]


  - (Training)   ppl: 0.2609993055462837, Accuracy: 0.019538216517165426, elapse: 0.15719339450200398 min


  - (Validation) ppl: 0.2308492213487625, Accuracy: 0.03407444598918856, elapse: 0.011979877948760986 min
[ Epoch 1 ]


  - (Training)   ppl: 0.17661265581846236, Accuracy: 0.1370466474210063, elapse: 0.1650582234064738 min


  - (Validation) ppl: 0.10538044944405556, Accuracy: -0.028638279819761588, elapse: 0.012109478314717611 min


In [8]:
valid_losss, train_losses, valid_accus, train_accus = train_validation.continue_training(params['save_model_path'], params['epoch'], model, train_dataloader, eval_dataloader, optimizer, device, criterion)

Checkpoint found and loaded - Resuming training
[ Epoch 2 ]


  - (Training)   ppl: 0.14249686151742935, Accuracy: 0.20002837834907883, elapse: 0.18731647729873657 min


  - (Validation) ppl: 0.10182568430900574, Accuracy: -0.05117330838421105, elapse: 0.012834278742472331 min
[ Epoch 3 ]


  - (Training)   ppl: 0.10645390748977661, Accuracy: 0.31931252712171926, elapse: 0.22762589454650878 min


  - (Validation) ppl: 0.08408648520708084, Accuracy: 0.10670699137871208, elapse: 0.019465947151184083 min
